In [2]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    #Camada que recebe todos os dados
    layers.Input(shape=(28,28,3)), # 28x28x1
    
    #Tratar a imagem
    layers.Resizing(28,28),
    layers.Rescaling(1./255),
    
    #Limpar da imagem
    layers.Conv2D(10, kernel_size=(3,3), strides=(1,1), padding='same'), # 10 @ 28x28
    layers.MaxPooling2D((2,2)),
    
    #Achatar 
    layers.Flatten(), # 7840
    
    #Análise da imagem
    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()), # 64
    
    layers.Dropout(0,2), #Desativa 20% dos neurônios apenas durante o treinamento
    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()), # 64
    layers.Dense(24, activation=activations.softmax, kernel_initializer=initializers.RandomNormal()) # 24
])

In [3]:
from tensorflow.keras import optimizers, losses, metrics

#Taxa de aprendizagem
lr = 0.001

model.compile(
    #Adam vai tentar não oscilar muito os valores
    optimizer = optimizers.Adam(
        learning_rate = lr    
    ),
    #Calcula a perda 
    loss = losses.SparseCategoricalCrossentropy(),
    #Métricas 
    metrics = [ metrics.sparse_categorical_accuracy ]
)

In [9]:
from tensorflow.keras import utils

path = './Data'
#Quantidade de dados dentro de cada batch
batch_size = 100


train = utils.image_dataset_from_directory(
    #caminho da imagem
    directory=path + '/Data/Train',
    #sim, vai embaralhar
    shuffle = True,
    #sempre embaralha a imagem da mesma forma
    seed = 1,
    subset = 'training',
    validation_split = 0.1,
    image_size = (28,28),
    #Em uma epoch, ele treina todos os batches, batches são as quantidades de repartições nos dados
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    directory=path + '/Data/Test',
    shuffle = True,
    seed = 1,
    subset = 'validation',
    validation_split = 0.1,
    image_size = (28,28),
    batch_size = batch_size
)

Found 50000 files belonging to 10 classes.
Using 45000 files for training.
Found 10000 files belonging to 10 classes.
Using 1000 files for validation.


In [10]:
from tensorflow.keras import callbacks

#Tolerância de erro, se ele piorar 5 vezes seguidas, ele para de tentar
patience = 5
#Número de épocas
epochs = 100

model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    #Nos diz seus resultados
    verbose = True,
    
    callbacks = [
        #Para que ele pare quando atingir sua paciência
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = True
        )
        #Salvando o modelo
        # callbacks .ModelCheckpoint(
        #     filepath = model_path,
        #Salvando os filtros
        #     save_weights_only = False,
        #     monitor = 'loss',
        #O melhor é o que possui menor perda
        #     mode = 'min',
        #     save_best_only = True
        # )
    ]
)

Epoch 1/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 52s 115ms/step - loss: 1.8394 - sparse_categorical_accuracy: 0.3357 - val_loss: 1.6959 - val_sparse_categorical_accuracy: 0.4050
Epoch 2/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - loss: 1.5996 - sparse_categorical_accuracy: 0.4241 - val_loss: 1.4991 - val_sparse_categorical_accuracy: 0.4630
Epoch 3/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - loss: 1.4701 - sparse_categorical_accuracy: 0.4740 - val_loss: 1.4161 - val_sparse_categorical_accuracy: 0.4920
Epoch 4/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - loss: 1.3949 - sparse_categorical_accuracy: 0.5024 - val_loss: 1.3483 - val_sparse_categorical_accuracy: 0.5180
Epoch 5/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - loss: 1.3341 - sparse_categorical_accuracy: 0.5244 - val_loss: 1.3042 - val_sparse_categorical_accuracy: 0.5340
Epoch 6/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - loss: 1.2787 - sparse_categorical_accuracy: 0.5466 - val_loss: 1.2669 - val_sparse_categorical_accurac

In [ ]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    layers.Input(shape=(28,28,3)), 
    
    layers.Resizing(28,28),
    layers.Rescaling(1./255),
    layers.RandomRotation((-0.2, 0.2)),

    layers.Conv2D(10, kernel_size=(3,3), strides=(1,1), padding='same'),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),

    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),
    layers.Dropout(0.2),
    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),
    layers.Dense(24, activation=activations.softmax, kernel_initializer=initializers.RandomNormal())
])

from tensorflow.keras import optimizers, losses, metrics

lr = 0.001

model.compile(
    optimizer = optimizers.Adam(
        learning_rate = lr    
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ metrics.sparse_categorical_accuracy ]
)

from tensorflow.keras import callbacks

patience = 5
epochs = 100
model_path = "./model.keras"

model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    verbose = True,
    
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks .ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)